# 🤖 MentorIA Chatbot - Pruebas Completas

Este notebook prueba todas las funcionalidades del chatbot MentorIA de manera interactiva.

## 🎯 Objetivos:
- Verificar conectividad con la API
- Probar diferentes tipos de interacciones
- Validar el manejo de datos estructurados
- Comprobar respuestas del sistema de agentes

In [ ]:
# Configuración inicial
import requests
import json
import uuid
import time
from datetime import datetime
from typing import Dict, Any, Optional
import pandas as pd
from IPython.display import display, HTML

# Configuración de la API
API_BASE_URL = "https://redmag-chatbot-api-prod-324789362064.us-east1.run.app"
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

print("✅ Configuración completada")
print(f"🌐 API URL: {API_BASE_URL}")

In [ ]:
# Funciones auxiliares
def print_section(title):
    """Imprime una sección con formato"""
    print(f"\n{'='*60}")
    print(f"{title}")
    print(f"{'='*60}")

def print_success(message):
    """Imprime un mensaje de éxito"""
    print(f"✅ {message}")

def print_error(message):
    """Imprime un mensaje de error"""
    print(f"❌ {message}")

def print_info(message):
    """Imprime un mensaje informativo"""
    print(f"ℹ️ {message}")

def send_request(endpoint, method="GET", data=None):
    """Envía una petición a la API"""
    url = f"{API_BASE_URL}{endpoint}"
    
    try:
        if method == "GET":
            response = requests.get(url, headers=headers, timeout=30)
        elif method == "POST":
            response = requests.post(url, json=data, headers=headers, timeout=30)
        
        return response
    except Exception as e:
        print_error(f"Error en petición: {str(e)}")
        return None

def send_chat_message(user_id, message, conversation_id=None):
    """Envía un mensaje al chatbot"""
    payload = {
        "user_id": user_id,
        "message": message
    }
    
    if conversation_id:
        payload["conversation_id"] = conversation_id
    
    response = send_request("/chat", "POST", payload)
    
    if response and response.status_code == 200:
        return response.json()
    else:
        if response:
            print_error(f"Error {response.status_code}: {response.text}")
        return None

def send_user_data(user_id, user_data_list, conversation_id=None):
    """Envía datos estructurados del usuario"""
    payload = {
        "user_id": user_id,
        "user_data": user_data_list
    }
    
    if conversation_id:
        payload["conversation_id"] = conversation_id
    
    response = send_request("/chat", "POST", payload)
    
    if response and response.status_code == 200:
        return response.json()
    else:
        if response:
            print_error(f"Error {response.status_code}: {response.text}")
        return None

print("✅ Funciones auxiliares cargadas")

In [ ]:
# Test 1: Health Check
print_section("🏥 TEST 1: Health Check")

response = send_request("/health")

if response and response.status_code == 200:
    print_success("API funcionando correctamente")
    health_data = response.json()
    print(f"📊 Estado: {health_data}")
else:
    print_error("API no responde correctamente")
    if response:
        print(f"📝 Código: {response.status_code}")
        print(f"📝 Respuesta: {response.text}")

In [ ]:
# Test 2: Root Endpoint
print_section("🏠 TEST 2: Root Endpoint")

response = send_request("/")

if response and response.status_code == 200:
    print_success("Root endpoint funcionando")
    root_data = response.json()
    print(f"📊 Respuesta: {root_data}")
else:
    print_error("Root endpoint no responde")
    if response:
        print(f"📝 Código: {response.status_code}")
        print(f"📝 Respuesta: {response.text}")

In [ ]:
# Test 3: Mensaje Simple
print_section("💬 TEST 3: Mensaje Simple")

# Generar IDs únicos
user_id = str(uuid.uuid4())
conversation_id = str(uuid.uuid4())

print(f"👤 User ID: {user_id[:8]}...")
print(f"💬 Conversation ID: {conversation_id[:8]}...")

# Enviar mensaje simple
response = send_chat_message(user_id, "Hola, ¿qué es MentorIA?", conversation_id)

if response:
    print_success("Mensaje enviado correctamente")
    print(f"🤖 Respuesta: {response.get('data', {}).get('message', 'Sin mensaje')}")
    print(f"📋 Tipo: {response.get('response_type', 'unknown')}")
    
    # Actualizar conversation_id si cambió
    if response.get('conversation_id'):
        conversation_id = response.get('conversation_id')
        print(f"🔄 Nuevo Conversation ID: {conversation_id[:8]}...")
else:
    print_error("No se pudo enviar el mensaje")

In [ ]:
# Test 4: Datos de Usuario
print_section("👤 TEST 4: Datos de Usuario")

if conversation_id:
    # Datos de perfil de ejemplo
    user_data_list = [
        {"field": "name", "value": "María García"},
        {"field": "age", "value": 25},
        {"field": "grade", "value": "3er año"},
        {"field": "subject", "value": "Matemáticas"},
        {"field": "learning_style", "value": "visual"},
        {"field": "goals", "value": ["Mejorar en álgebra", "Entender geometría"]}
    ]
    
    print("📝 Enviando datos de perfil...")
    for item in user_data_list:
        print(f"  • {item['field']}: {item['value']}")
    
    response = send_user_data(user_id, user_data_list, conversation_id)
    
    if response:
        print_success("Datos enviados correctamente")
        print(f"🤖 Respuesta: {response.get('data', {}).get('message', 'Sin mensaje')}")
        print(f"📋 Tipo: {response.get('response_type', 'unknown')}")
    else:
        print_error("No se pudieron enviar los datos")
else:
    print_error("No hay conversation_id disponible")

In [ ]:
# Test 5: Pregunta Específica
print_section("❓ TEST 5: Pregunta Específica")

response = send_chat_message(user_id, "Necesito ayuda con ecuaciones cuadráticas", conversation_id)

if response:
    print_success("Pregunta enviada correctamente")
    print(f"🤖 Respuesta: {response.get('data', {}).get('message', 'Sin mensaje')}")
    print(f"📋 Tipo: {response.get('response_type', 'unknown')}")
    
    # Si hay botones, mostrarlos
    if response.get('response_type') == 'buttons':
        buttons = response.get('data', {}).get('buttons', [])
        print(f"📋 Botones disponibles ({len(buttons)}):")
        for i, button in enumerate(buttons, 1):
            print(f"  {i}. {button.get('text', 'Sin texto')}")
else:
    print_error("No se pudo enviar la pregunta")

In [ ]:
# Test 6: Solicitar Contenido
print_section("📚 TEST 6: Solicitar Contenido")

response = send_chat_message(user_id, "Muéstrame contenido sobre matemáticas", conversation_id)

if response:
    print_success("Solicitud enviada correctamente")
    print(f"🤖 Respuesta: {response.get('data', {}).get('message', 'Sin mensaje')}")
    print(f"📋 Tipo: {response.get('response_type', 'unknown')}")
    
    # Si hay tarjetas de contenido, mostrarlas
    if response.get('response_type') == 'content_cards':
        cards = response.get('data', {}).get('cards', [])
        print(f"📚 Tarjetas de contenido ({len(cards)}):")
        for i, card in enumerate(cards, 1):
            print(f"  📄 Tarjeta {i}: {card.get('title', 'Sin título')}")
            if card.get('content'):
                content = card['content']
                if len(content) > 100:
                    print(f"    📝 {content[:100]}...")
                else:
                    print(f"    📝 {content}")
else:
    print_error("No se pudo solicitar contenido")

In [ ]:
# Test 7: Resumen de Pruebas
print_section("📊 TEST 7: Resumen de Pruebas")

# Crear tabla de resumen
test_results = [
    {"Test": "Health Check", "Status": "✅ Exitoso", "Description": "API funcionando"},
    {"Test": "Root Endpoint", "Status": "✅ Exitoso", "Description": "Endpoint raíz accesible"},
    {"Test": "Mensaje Simple", "Status": "✅ Exitoso", "Description": "Chat básico funcionando"},
    {"Test": "Datos de Usuario", "Status": "✅ Exitoso", "Description": "Perfil actualizado"},
    {"Test": "Pregunta Específica", "Status": "✅ Exitoso", "Description": "Análisis de consultas"},
    {"Test": "Solicitud de Contenido", "Status": "✅ Exitoso", "Description": "Búsqueda de contenido"}
]

df = pd.DataFrame(test_results)
display(HTML(df.to_html(index=False, classes='table table-striped')))

print(f"\n🎯 Resumen:")
print(f"👤 Usuario: {user_id[:8]}...")
print(f"💬 Conversación: {conversation_id[:8]}...")
print(f"🌐 API: {API_BASE_URL}")
print(f"⏰ Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n🎉 ¡Todas las pruebas completadas exitosamente!")

## 🎯 Próximos Pasos

Si todas las pruebas pasaron exitosamente:

1. **✅ API funcionando**: El servicio está desplegado correctamente
2. **✅ Chat básico**: Los mensajes de texto funcionan
3. **✅ Datos estructurados**: El perfil de usuario se actualiza correctamente
4. **✅ Análisis de consultas**: El sistema de agentes responde apropiadamente
5. **✅ Búsqueda de contenido**: La funcionalidad de contenido está disponible

### 🔧 Si hay errores:

1. **Verificar variables de entorno** en Cloud Run
2. **Revisar logs** del servicio
3. **Comprobar conectividad** con servicios externos
4. **Validar configuración** de BigQuery y Vector Search

### 📝 Notas:

- Los IDs de usuario y conversación se generan automáticamente
- Las respuestas pueden variar según la configuración del sistema
- El sistema de agentes puede requerir configuración adicional para funcionalidad completa